In [3]:
import sys
# sys.path.append('/home/czl/workspace/LocAgent')
sys.path.append('/home/azureuser/cloudfiles/code/Users/v-xhou/locagent/LocAgent')

from evaluation.eval_metric import evaluate_results
level2key_dict = {
    'file': 'found_files',
    'module': 'found_modules',
    'function': 'found_entities',
}

/home/azureuser/cloudfiles/code/Users/v-xhou/locagent/LocAgent/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# eval with dataset
locagent_loc_file = '../results/GPT4-1-mini0414/loc_outputs.jsonl'
locagent_res = evaluate_results(locagent_loc_file,
                        level2key_dict,
                        dataset='czlll/SWE-bench_Lite', split='test', 
                        metrics=['acc'],
                        # metrics=['ndcg'],
                        )
locagent_res

file                 module         function        
    Acc@1   Acc@3   Acc@5  Acc@5  Acc@10    Acc@5  Acc@10
0  0.6569  0.8139  0.8212  0.354  0.4453   0.2518  0.3175

In [8]:
# eval with dataset
locagent_loc_file = '../results/GPT4-1-mini0101/loc_outputs.jsonl'
locagent_res = evaluate_results(locagent_loc_file,
                        level2key_dict,
                        dataset='czlll/SWE-bench_Lite', split='test', 
                        metrics=['acc'],
                        # metrics=['ndcg'],
                        )
locagent_res

file                 module         function        
    Acc@1   Acc@3   Acc@5  Acc@5  Acc@10    Acc@5  Acc@10
0  0.6788  0.8175  0.8248  0.354  0.4562    0.281  0.3577

In [5]:
# eval with dataset
locagent_loc_file = '../results/GPT4-1/loc_outputs.jsonl'
locagent_res = evaluate_results(locagent_loc_file,
                        level2key_dict,
                        dataset='czlll/SWE-bench_Lite', split='test', 
                        metrics=['acc'],
                        # metrics=['ndcg'],
                        )
locagent_res

file                  module         function        
    Acc@1   Acc@3   Acc@5   Acc@5  Acc@10    Acc@5  Acc@10
0  0.7153  0.8394  0.8467  0.3723  0.4453   0.2956  0.3431

In [1]:
import os
import json
from pathlib import Path

# Set up paths
bm25_index_path = Path("../index_data/SWE-bench_Lite_bm25_13K/BM25_index")
graph_index_path = Path("../index_data/SWE-bench_Lite_bm25_13K/graph_index_v2.3")
loc_outputs_path = Path("../results/GPT4-1/loc_outputs.jsonl")

# Count unique folders (directories) in BM25 index
bm25_folders = set()
if bm25_index_path.exists():
    for item in bm25_index_path.iterdir():
        if item.is_dir():
            bm25_folders.add(item.name)
    print(f"Number of unique folders in BM25 index: {len(bm25_folders)}")
    print(f"Sample BM25 folders: {list(bm25_folders)[:5]}")
else:
    print(f"BM25 index path not found: {bm25_index_path}")
    # Try to find the correct path
    parent_path = Path("..")
    print(f"Contents of parent directory: {list(parent_path.iterdir())[:10]}")

# Count unique files in graph index
graph_files = set()
if graph_index_path.exists():
    for item in graph_index_path.iterdir():
        if item.is_file():
            graph_files.add(item.name)
        elif item.is_dir():
            # Also check subdirectories
            for subitem in item.rglob("*"):
                if subitem.is_file():
                    graph_files.add(subitem.relative_to(graph_index_path))
    print(f"Number of unique files in graph index: {len(graph_files)}")
else:
    print(f"Graph index path not found: {graph_index_path}")

# Count unique instance IDs in loc_outputs.jsonl
unique_instance_ids = set()
if loc_outputs_path.exists():
    with open(loc_outputs_path, 'r') as f:
        for line in f:
            if line.strip():  # Skip empty lines
                try:
                    data = json.loads(line)
                    if 'instance_id' in data:
                        unique_instance_ids.add(data['instance_id'])
                except json.JSONDecodeError as e:
                    print(f"Error parsing line: {e}")
    print(f"Number of unique instance IDs in loc_outputs.jsonl: {len(unique_instance_ids)}")
    
    # Show first few instance IDs as a sample
    print(f"Sample instance IDs: {list(unique_instance_ids)[:5]}")
else:
    print(f"loc_outputs.jsonl not found at: {loc_outputs_path}")

# Summary
print("\n=== Summary ===")
print(f"BM25 index files: {len(bm25_folders)}")
print(f"Graph index files: {len(graph_files)}")
print(f"Unique instance IDs in loc_outputs: {len(unique_instance_ids)}")

Number of unique folders in BM25 index: 300
Sample BM25 folders: ['scikit-learn__scikit-learn-13779', 'django__django-16595', 'django__django-12113', 'django__django-14411', 'scikit-learn__scikit-learn-25500']
Number of unique files in graph index: 300
Number of unique instance IDs in loc_outputs.jsonl: 275
Sample instance IDs: ['scikit-learn__scikit-learn-13779', 'django__django-16595', 'django__django-12113', 'django__django-14411', 'scikit-learn__scikit-learn-25500']

=== Summary ===
BM25 index files: 300
Graph index files: 300
Unique instance IDs in loc_outputs: 275


In [12]:
from datasets import load_dataset
import pandas as pd

# Load both datasets
print("Loading datasets...")
dataset1 = load_dataset("czlll/SWE-bench_Lite", split="test")
dataset2 = load_dataset("princeton-nlp/SWE-bench_Lite_bm25_13K", split="test")

# Create dictionaries mapping instance_id to problem_statement
problems1 = {item['instance_id']: item.get('problem_statement', '') for item in dataset1}
problems2 = {item['instance_id']: item.get('problem_statement', '') for item in dataset2}

# Get instance ID sets
ids1 = set(problems1.keys())
ids2 = set(problems2.keys())

print(f"\nDataset sizes:")
print(f"czlll/SWE-bench_Lite: {len(ids1)} instances")
print(f"princeton-nlp/SWE-bench_Lite_bm25_13K: {len(ids2)} instances")

# Calculate instance ID overlap
id_overlap = ids1.intersection(ids2)
print(f"\n=== Instance ID Overlap ===")
print(f"Common instance IDs: {len(id_overlap)}")
print(f"Jaccard similarity (IDs): {len(id_overlap) / len(ids1.union(ids2)):.3f}")

# Check problem statement matches for common instances
matching_problems = 0
mismatched_problems = []

for instance_id in id_overlap:
    if problems1[instance_id] == problems2[instance_id]:
        matching_problems += 1
    else:
        mismatched_problems.append(instance_id)

print(f"\n=== Problem Statement Comparison (for common instances) ===")
print(f"Instances with matching problem statements: {matching_problems}/{len(id_overlap)} ({matching_problems/len(id_overlap)*100:.1f}%)")
print(f"Instances with different problem statements: {len(mismatched_problems)}")

if mismatched_problems:
    print(f"\nFirst 5 instances with different problem statements:")
    for instance_id in mismatched_problems[:5]:
        print(f"\n- {instance_id}")
        print(f"  czlll length: {len(problems1[instance_id])} chars")
        print(f"  princeton length: {len(problems2[instance_id])} chars")
        
        # Show a snippet of the difference
        if problems1[instance_id] and problems2[instance_id]:
            print(f"  czlll snippet: {problems1[instance_id][:100]}...")
            print(f"  princeton snippet: {problems2[instance_id][:100]}...")

# Summary
print(f"\n=== SUMMARY ===")
if len(id_overlap) == len(ids1) == len(ids2) and matching_problems == len(id_overlap):
    print("✓ Both datasets have identical instance IDs and problem statements!")
else:
    print("✗ Datasets differ in:")
    if len(ids1) != len(ids2):
        print(f"  - Number of instances: {len(ids1)} vs {len(ids2)}")
    if len(id_overlap) < min(len(ids1), len(ids2)):
        print(f"  - Instance IDs (only {len(id_overlap)} common)")
    if matching_problems < len(id_overlap):
        print(f"  - Problem statements ({len(mismatched_problems)} differ)")

# Create a simple comparison report
report = {
    'total_instances_czlll': len(ids1),
    'total_instances_princeton': len(ids2),
    'common_instance_ids': len(id_overlap),
    'jaccard_similarity': len(id_overlap) / len(ids1.union(ids2)),
    'matching_problem_statements': matching_problems,
    'mismatched_problem_statements': len(mismatched_problems)
}

print(f"\n{report}")

Loading datasets...

Dataset sizes:
czlll/SWE-bench_Lite: 274 instances
princeton-nlp/SWE-bench_Lite_bm25_13K: 300 instances

=== Instance ID Overlap ===
Common instance IDs: 274
Jaccard similarity (IDs): 0.913

=== Problem Statement Comparison (for common instances) ===
Instances with matching problem statements: 273/274 (99.6%)
Instances with different problem statements: 1

First 5 instances with different problem statements:

- mwaskom__seaborn-2848
  czlll length: 965 chars
  princeton length: 5820 chars
  czlll snippet: pairplot fails with hue_order not containing all hue values in seaborn 0.11.1
In seaborn < 0.11, one...
  princeton snippet: PairGrid errors with `hue` assigned in `map`
In seaborn version 0.9.0 I was able to use the followin...

=== SUMMARY ===
✗ Datasets differ in:
  - Number of instances: 274 vs 300
  - Problem statements (1 differ)

{'total_instances_czlll': 274, 'total_instances_princeton': 300, 'common_instance_ids': 274, 'jaccard_similarity': 0.913333333333